In [7]:
import sqlalchemy
from sqlalchemy import create_engine, inspect, func
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
Base= automap_base()
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta
import numpy as np
from flask import Flask, jsonify

In [8]:
database_path='C:\workspace\Resources\hawaii.sqlite'
engine=create_engine(f'sqlite:///{database_path}')
conn= engine.connect()
Base.prepare(engine, reflect= True)
Base.classes.keys()
Measurement= Base.classes.measurement
Station= Base.classes.station

In [14]:
session=Session(engine)

In [18]:
station_list= session.query(Station.station).all()
station_counts=[Station.station, func.count(Measurement.tobs)]
twelve_month_ago = dt.date(2017,8,23) - relativedelta(months=+12)
now= dt.date(2017,8,23)
most_active=session.query(*station_counts).\
    filter(Station.station==Measurement.station).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.tobs).desc()).\
    limit(1).all()
most_active_unzip = list(np.ravel(most_active))
temp_12=session.query(Measurement.date, Measurement.tobs).\
    filter(Station.station==Measurement.station).\
    filter(Station.station==most_active_unzip[0]).\
    filter(Measurement.date <=now).\
    filter(Measurement.date >= twelve_month_ago).all()

In [19]:
app= Flask(__name__)

In [20]:
@app.route("/")
def home():
    return (f"Welcome to Surfs Up!<br/>"
            f"Available route:<br/>"
            f"/api/v1.0/precipitation<br/>"
            f"/api/v1.0/stations<br/>"
            f"/api/v1.0/tobs")

@app.route("/api/v1.0/precipitation")
def precipitation():
    session= Session(engine)
#     twelve_month_ago = dt.date(2017,8,23) - relativedelta(months=+12)
#     now= dt.date(2017,8,23)
    prcp_12=session.query(Measurement.date, Measurement.prcp).\
        filter(Measurement.date <=now).\
        filter(Measurement.date >= twelve_month_ago).all()
    session.close()
    all_prcp=[]
    for date, prcp in prcp_12:
        prcp_dict={}
        prcp_dict["date"]=date
        prcp_dict["prcp"]=prcp
        all_prcp.append(prcp_dict)
    return jsonify(all_prcp)

@app.route("/api/v1.0/stations")
def stations():
#     session= Session(engine)
#     station_list= session.query(Station.station).all()
#     session.close()
    return jsonify(station_list)

@app.route("/api/v1.0/tobs")
def temps():
    return jsonify(temp_12)

# # @app.route("/api/v1.0/<start>")
# # def start_date(start):

# @app.route("/api/v1.0/<start>/<end>")
# def start_end():
#     session= Session(engine)
#     start=dt.date(start)
#     end=dt.date(end)
#     session = Session(engine)
#     sel=[func.min(Measurement.tobs),func.max(Measurement.tobs),func.avg(Measurement.tobs)]
#     query_para=session.query(*sel).\
#                 filter(Station.station==Measurement.station).\
#                 filter(Station.station==most_active_unzip[0]).\
#                 filter(Measurement.date <=end).\
#                 filter(Measurement.date >=start).all()
#     session.close()
#     return jsonify(query_para)

if __name__=="__main__":
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1